### Gradio

In [ ]:
!pip install keras-nlp keras langchainhub langchain langchain-google-vertexai

In [ ]:

!pip install -q -U keras-nlp
!pip install -q -U keras>=3
!pip install -U gradio



In [ ]:
import keras_hub
import gradio as gr
base = keras_hub.models.GemmaCausalLM.from_preset("kaggle://favouryahdii/gemma-nutritionx/keras/gemma-nutritionx-2b")
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
def process_user_input(user_input, api_data="", max_length=512):
    # Preprocess the input
    processed_input = template.format(instruction=user_input, response="")

    # Assuming the model (base) is already defined and loaded
    prediction = base.generate(processed_input, max_length)  # Adjust based on your model's predict method
    return prediction.split("Response:\n")[1].strip()

# Step 5: Handle user input, model response, and function calling
def handle_function_call(user_input, history):
    history_list = []
    for msg in history:
        history_list.append(msg)
    history_list.append(user_input)

    # Always generate a model response first
    model_response = process_user_input(user_input)

    return model_response




In [ ]:
import gradio as gr

# Create a Gradio interface
gr.ChatInterface(handle_function_call, type="messages").launch(debug=True)

### Eval

In [1]:
!pip install huggingface_hub pandas tqdm -q

In [102]:
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)

# Test your LLM client
#llm_client.text_generation(prompt="How are you today?", max_new_tokens=20)

In [103]:
# lets test the models performance
import json
with open('sampled.json', 'r') as file:
    train = json.load(file)

with open('unsampled.json', 'r') as file:
    test = json.load(file)



In [104]:
import re
import pandas as pd
from tqdm.auto import tqdm
from huggingface_hub import InferenceClient, notebook_login

tqdm.pandas()  # load tqdm's pandas support
pd.set_option("display.max_colwidth", None)

notebook_login()

In [105]:
JUDGE_PROMPT = """
You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer as an integer on a scale of 1 to 4, where 1 means that the system_answer is not helpful at all, and 4 means that the answer completely and helpfully addresses the question.

Provide your feedback as follows:

Feedback:::
Total rating: (your rating, as an integer between 1 and 4)

Now here are the question and answer.

Question: {question}
Answer: {answer}

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.
Feedback:::
Total rating: """

In [106]:
#separate test set into two columns based on Instruction/ Response
import pandas as pd
eval_set = pd.read_csv('human_rating.csv')

In [107]:
eval_set.head()

,question,model_answer,rating,explanation
0,What can be considered a high protein breakfast?,You could consider having eggs with whole wheat toast and avocado for breakfast.,3,Answer could be more detailed
1,I am a 28 year old female who just finished 400 Meter race and has no appetite for real food. What can I take?,"You can take Imodium, which is a medication that helps relieve diarrhea.",1,answer is non-factual
2,I am a 28 year old female who just finished 400 Meter race and has no appetite for real food. What can I eat?,"Some fruits and vegetables may be beneficial, but it is important to prioritize nutrient-dense foods that will provide calories and nutrients to help rebuild muscle and strength.",3,Answer could be more specific
3,What is a good desert for a high protein supper?,"A good desert for a high protein supper might include dairy products such as yogurt or cheese, along with fruit such as berries or apples.",3,Answer could be more detailed
4,An example of a high protein lunch is?,An example of a high protein lunch is chicken salad sandwich on whole-wheat bread with salad dressing on the side.,2,Nobody eats salad dressing with sandwich


In [108]:
# eval set, delete iloc 21 from the set
eval_set = eval_set.drop(21)


In [109]:

eval_set["llm_judge"] = eval_set.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(question=x["question"], answer=x["model_answer"]),
        max_new_tokens=1000,
    ),
    axis=1,
)

  0%|          | 0/29 [00:00<?, ?it/s]

In [110]:
def extract_judge_score(answer: str, split_str: str = "Total rating:") -> int:
    try:
        if split_str in answer:
            rating = answer.split(split_str)[1]
        else:
            rating = answer
        digit_groups = [el.strip() for el in re.findall(r"\d+(?:\.\d+)?", rating)]
        return float(digit_groups[0])
    except Exception as e:
        print(e)
        return None


eval_set["llm_judge_score"] = eval_set["llm_judge"].apply(extract_judge_score)
# Rescale the score given by the LLM on the same scale as the human score
#eval_set["llm_judge_score"] = (eval_set["llm_judge_score"] / 4) + 1

In [111]:
print("Correlation between LLM-as-a-judge and the human raters:")
print(f"{eval_set['llm_judge_score'].corr(eval_set['rating'], method='pearson'):.3f}")

Correlation between LLM-as-a-judge and the human raters:
0.088


In [ ]:
# LLM as a judge is not able to assess nuances in responses like a human would.

### RAG


In [1]:
!pip install langchain-google-vertexai

In [101]:
!pip install langchain-community

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 427, in resolve
    failure_causes = self._attempt_to_pin_criterion(name)

In [3]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ["TAVILY_API_KEY"] = "KEY"
search = TavilySearchResults()

search.invoke('What are the grocery stores near the university of Leeds?')

[{'url': 'https://medium.com/leeds-beckett-university-international-study/where-to-find-the-best-international-groceries-in-leeds-6f17ed6cd27a',
  'content': 'Nong Fern Thai Supermarket. 2. Sultan Halal Market. For those seeking Arabic foods and spices, Sultan Halal Market is an ideal choice. Situated near the University of Leeds, this supermarket has a'},
 {'url': 'https://www.coop.co.uk/store-finder/LS2-9JZ/2-lifton-place',
  'content': 'Other food stores near UNIF Leeds Uni - Lifton Place. 1 Leeds - Burley Street 0.5 miles Open today until 11pm; 2 Leeds - Merrion Way 0.62 miles Open today until 10pm; 3 Leeds - Cardigan Road 0.93 miles Open today until Midnight; View all Co-op Food stores in West Yorkshire'},
 {'url': 'https://www.yelp.com/search?cflt=grocery&find_loc=Leeds,+West+Yorkshire',
  'content': "Best Grocery in Leeds, West Yorkshire, United Kingdom - Abu Bakar Supermarket, Sainsbury's, Aldi, CC Continental Supermarket, Waitrose & Partners, AAA Supermarket, Morrisons, Sainsb

In [4]:
!pip install keras-nlp

In [5]:
from langchain import hub
from langchain_google_vertexai import ChatVertexAI, GemmaChatLocalKaggle
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")

llm = GemmaChatLocalKaggle(model_name="kaggle://favouryahdii/gemma-nutritionx/keras/gemma-nutritionx-2b", keras_backend="jax")


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
from langchain_core.messages import (
    HumanMessage
)

message1 = HumanMessage(content="what is a balanced diet?")
answer1 = llm.invoke([message1], max_tokens=30)
print(answer1)

content='<start_of_turn>user\nwhat is a balanced diet?<end_of_turn>\n<start_of_turn>model\nwhat is a balanced diet? bourg\n Février 2018\n' additional_kwargs={} response_metadata={} id='run-713b41a7-29c8-43d9-bc21-6ba59ee0b6a9-0'


In [7]:
tools = [search]
agent = create_react_agent(llm, tools, prompt=prompt)

In [9]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

input_prompt = "What are the grocery stores near the university of Leeds?"
output = agent_executor.invoke({"input": input_prompt})



> Entering new AgentExecutor chain...
<start_of_turn>user
Answer the following questions as best you can. You have access to the following tools:

tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [tavily_search_results_json]
Action Input: the input to the actionthe action to take, should be one of [tavily_search_results_json] is not a valid tool, try one of [tavily_search_results_json].<start_of_turn>user
Answer the following questions as best you can. You have access to the following tools:

tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input